In [ ]:
import numpy as np
from lammps import PyLammps
import tkinter as tk
from moduleGA import create_chain

In [ ]:
import tkinter as tk
my_w=tk.Tk()
my_w.geometry('200x50')

series = []
ones = []
twos = []
def take_input(value):
    tmp = value
    if tmp == 1:
        
        ones.append(tmp)
        if len(ones) == 45:
            print('\nYou have 5 ones left\n')
        elif len(ones) == 50:
            print('\nYou have no ones left\n')
        elif len(ones) > 50:
            raise ValueError('You cannot have more than 50 ones')
            
    elif tmp == 2:
        
        twos.append(tmp)
        if len(twos) == 45:
            print('\nYou have 5 twos left\n')
        elif len(twos) == 50:
            print('\nYou have no twos left\n')
        elif len(twos) > 50:
            raise ValueError('You cannot have more than 50 twos')
    
    series.append(tmp)
    if len(series)%10 == 0:
        print(f'{100-len(series)} beads left')
    if len(series) == 100:
        raise ValueError('You cannot have more than 100 beads')
    print(value)
b1=tk.Button(my_w,text='1',width=10, command=lambda *args: take_input(1))
b1.grid(row=1,column=1)
b2=tk.Button(my_w,text='2',width=10, command=lambda *args: take_input(2))
b2.grid(row=1,column=2)
my_w.mainloop()

In [ ]:
chain = [str(elem) for elem in series]
with open('start.data', 'r+') as f:
        lines = f.readlines()

changes = []
for elem in lines[17:117]:
    changes.append(elem.split())

for i in range(len(changes)):
    changes[i][2] = chain[i]
    changes[i] = ' '.join(changes[i])

with open('start_new.data','w') as f:
    for elem in lines[:17]:
        f.write(elem)
    for elem in changes:
        f.write(elem + '\n')
    for elem in lines[117:]:
        f.write(elem)

print('You have successfully created your chain')

In [ ]:
L = PyLammps()

In [ ]:
L.command('units lj')
L.command('dimension 3')
L.command('atom_style full')
L.command('bond_style fene')
L.command('pair_style lj/cut 2.5')

#### Read Data

In [ ]:
L.command('read_data start_new.data')

#### Bond

In [ ]:
L.command('bond_coeff 1 30.00 1.50 1.00 1.00')

#### Pair Interaction

In [ ]:
L.command('pair_coeff 1 1 1.0 1.0 2.5')
L.command('pair_coeff 2 2 0.2 1.0 2.5')
L.command('pair_coeff 1 2 0.2 1.0 2.5')

#### Group Define

In [ ]:
L.command('special_bonds fene')
L.command('pair_modify shift yes')

#### Velocity

In [ ]:
L.command('velocity all create 1.0 484733 mom yes rot yes dist gaussian')

#### Minimise & Fix

In [ ]:
L.command('minimize 1e-4 1e-6 100 1000')

In [ ]:
L.command('fix 1 all nve')
L.command('fix 2 all langevin 1.0 1.0 100.0 48279')
L.command('fix 3 all momentum 100 linear 1 1 1')

#### Equilibration Run

In [ ]:
L.command('run_style verlet')
L.command('timestep 0.005')
L.command('thermo 100000')
L.command('thermo_style custom step temp press pe epair')
L.command('run 1000000')

#### Production Run

In [ ]:
L.command('compute 1 all gyration')
L.command('fix RgAve all ave/time 1000 1000 1000000 c_1 file AvgRg.data')
L.command('dump 1 all custom 10000 FENE.lammpstrj id mol type xu yu zu vx vy vz')
L.command('run 1000000')

In [ ]:
rg = np.loadtxt('AvgRg.data')[1]
print(f'The radius of gyration is {rg}')